<a href="https://colab.research.google.com/github/barvaliyavishal/Movie-Recommendation-System-Using-Python-and-Spark/blob/main/Movie_Recommendation_Engine_Using_PySpark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dataset Used : [Dataset Link](https://www.youtube.com/redirect?event=video_description&redir_token=QUFFLUhqbTF2clhUUjVnSjF4VzhMYjRTX0l4RmJjN01uQXxBQ3Jtc0ttXzNKcXlwcGN5VWFtdnRxZllCNG9VU3F1TWZsMDcxVnBhVVJDV1Q4RkxJZ2ppbjZmNWhsSTFNcHo4T1pzaW1MVTlEd3h0Skt5Q2c4aTV1Y1FjUTM2WEFGbF92ZTczWXB4bzRXSlBkMUIyb21BaEQ1VQ&q=https%3A%2F%2Ffiles.grouplens.org%2Fdatasets%2Fmovielens%2Fml-25m.zip&v=eyEabQRBMQA)

### Importing needed Libraries

In [311]:
import pyspark
import pandas as pd
import re
from pyspark.sql import SparkSession

### Creating Spark Seesion

In [312]:
spark = SparkSession.builder.master("local[*]") \
                    .appName('Movie Recommendation Engine') \
                    .getOrCreate()

### Loading Movies DataSet into Spark DataFrame

In [313]:
movies = spark.read. \
      option("header","true"). \
      option("inferSchema","true"). \
      csv('/content/drive/MyDrive/Colab Notebooks/movies.csv')

Total number of Movies in dataset

In [314]:
movies.count()

62423

In [315]:
movies.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

### Cleaning movies titles using spark and regex 
##### This function will clean movie title by removing all special chars from title except space and alphanumeric char

In [316]:
def clean_title(title):
  
  return re.sub("[^a-zA-Z0-9 ]","",title)

In [317]:
 # Example 
 print(clean_title("Vishal--- ====@#$%barvaliya?"))

Vishal barvaliya


Before Cleaning

In [318]:
movies.filter(col('title').contains(')')).show(truncate=False)

+-------+-------------------------------------+-------------------------------------------+
|movieId|title                                |genres                                     |
+-------+-------------------------------------+-------------------------------------------+
|1      |Toy Story (1995)                     |Adventure|Animation|Children|Comedy|Fantasy|
|2      |Jumanji (1995)                       |Adventure|Children|Fantasy                 |
|3      |Grumpier Old Men (1995)              |Comedy|Romance                             |
|4      |Waiting to Exhale (1995)             |Comedy|Drama|Romance                       |
|5      |Father of the Bride Part II (1995)   |Comedy                                     |
|6      |Heat (1995)                          |Action|Crime|Thriller                      |
|7      |Sabrina (1995)                       |Comedy|Romance                             |
|8      |Tom and Huck (1995)                  |Adventure|Children               

In [319]:
from pyspark.sql.functions import col,udf
from pyspark.sql.types import StringType

In [320]:
clean_titles = udf(clean_title,StringType())

In [321]:
movies = movies.withColumn("title", clean_titles('title'))

After cleaning movie titles 

In [338]:
movies.filter(col('movieId') <= 20).sort('movieId').show(truncate=False)

+-------+----------------------------------+-------------------------------------------+
|movieId|title                             |genres                                     |
+-------+----------------------------------+-------------------------------------------+
|1      |Toy Story 1995                    |Adventure|Animation|Children|Comedy|Fantasy|
|2      |Jumanji 1995                      |Adventure|Children|Fantasy                 |
|3      |Grumpier Old Men 1995             |Comedy|Romance                             |
|4      |Waiting to Exhale 1995            |Comedy|Drama|Romance                       |
|5      |Father of the Bride Part II 1995  |Comedy                                     |
|6      |Heat 1995                         |Action|Crime|Thriller                      |
|7      |Sabrina 1995                      |Comedy|Romance                             |
|8      |Tom and Huck 1995                 |Adventure|Children                         |
|9      |Sudden Death

In [323]:
movies = movies.drop_duplicates(subset=['title'])

In [324]:
movies_df = movies.toPandas()
movies_df

,movieId,title,genres
0,172739,,(no genres listed)
1,173801,1972,Comedy
2,195899,1961,Comedy
3,196543,1978,Animation
4,193211,1965,(no genres listed)
...,...,...,...
62293,144736,unINDIAN 2015,Comedy|Romance
62294,128536,vegtigris 2001,Comedy
62295,200168,vertidskrig 2012,(no genres listed)
62296,5507,xXx 2002,Action|Crime|Thriller


In [325]:
movies_df  = movies_df[list(movies_df.columns[~movies_df.columns.duplicated()])]
movies_df

,movieId,title,genres
0,172739,,(no genres listed)
1,173801,1972,Comedy
2,195899,1961,Comedy
3,196543,1978,Animation
4,193211,1965,(no genres listed)
...,...,...,...
62293,144736,unINDIAN 2015,Comedy|Romance
62294,128536,vegtigris 2001,Comedy
62295,200168,vertidskrig 2012,(no genres listed)
62296,5507,xXx 2002,Action|Crime|Thriller


Using Vectorizer on title to create search engine first

In [326]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(ngram_range=(1,2))

tfidf = vectorizer.fit_transform(movies_df['title'])

Creating Search Function

In [327]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


In [328]:
def search(title):
  title = clean_title(title)
  query_vec = vectorizer.transform([title])
  similarity = cosine_similarity(query_vec,tfidf)
  indices = np.argpartition(similarity, -5)[-5:]
  res = movies_df.iloc[indices[0]][::-1][:5]
  return res

In [329]:
import ipywidgets as w
from IPython.display import display


In [330]:
movie_input = w.Text(
    description = "Movie Title:",
    disabled = False
)

movie_list = w.Output()
def on_type(data):
  with movie_list:
    movie_list.clear_output()
    title = data["new"]
    if len(title) > 5:
      display(search(title))

movie_input.observe(on_type,names='value')
display(movie_input,movie_list)

Text(value='', description='Movie Title:')

Output()

In [335]:
ratings = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/ratings.csv")
ratings

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


#Creating Recommendation System

In [332]:
def find_similar_movies(movieId):
  similar_users = ratings[(ratings['movieId'] == movieid) & (ratings['rating'] >= 4) ]['userId'].unique()
  similar_user_recs = ratings[(ratings['userId'].isin(similar_users) & (ratings['rating'] > 4))]['movieId']

  similar_user_recs = similar_user_recs.value_counts() / len(similar_users)
  similar_user_recs = similar_user_recs[similar_user_recs > .10]

  all_users = ratings[(ratings['movieId'].isin(similar_user_recs.index)) & (ratings['rating'] > 4)]
  all_users_recs = all_users['movieId'].value_counts() / len(all_users['userId'].unique())

  rec_percentages = pd.concat([similar_user_recs, all_users_recs],axis=1)
  rec_percentages.columns = ['similar','all']
  rec_percentages['score'] = rec_percentages['similar'] / rec_percentages['all']
  rec_percentages = rec_percentages.sort_values('score',ascending=False)
  return rec_percentages.head(10).merge(movies_df, left_index=True, right_on="movieId")[['score','title','genres']]

In [333]:
find_similar_movies(1)

,score,title,genres
11933,158.486182,Hulk 2003,Action|Adventure|Sci-Fi
12576,42.157324,Incredible Hulk The 2008,Action|Sci-Fi
54098,26.932292,SpiderMan 3 2007,Action|Adventure|Sci-Fi|Thriller|IMAX
41781,24.299781,Hancock 2008,Action|Adventure|Comedy|Crime|Fantasy
11211,22.377852,Hellboy 2004,Action|Adventure|Fantasy|Horror
24238,20.650733,Terminator 3 Rise of the Machines 2003,Action|Adventure|Sci-Fi
61915,20.111741,XMen Origins Wolverine 2009,Action|Sci-Fi|Thriller
44727,18.181971,King Kong 2005,Action|Adventure|Drama|Fantasy|Thriller
36298,17.868540,Chronicles of Riddick The 2004,Action|Sci-Fi|Thriller
28620,17.532425,Transporter The 2002,Action|Crime


Creating Widget Text Box in which you can enter any movie name you like and based on that movie name our recommendation engine will recommend top best other movies which you may like

In [334]:
movie_input_name = w.Text(
    Description = "Movie title",
    disabled = False
)
recommendation_list = w.Output()
def on_type(data):
  recommendation_list.clear_output()
  title = data['new']
  if len(title) > 5:
    res = search(title) 
    movie_id = res.iloc[0]["movieId"]
    display( find_similar_movies(movie_id))

movie_input_name.observe(on_type,names='value') 
display(movie_input_name, recommendation_list)


Text(value='')

Output()

,score,title,genres
11933,158.486182,Hulk 2003,Action|Adventure|Sci-Fi
12576,42.157324,Incredible Hulk The 2008,Action|Sci-Fi
54098,26.932292,SpiderMan 3 2007,Action|Adventure|Sci-Fi|Thriller|IMAX
41781,24.299781,Hancock 2008,Action|Adventure|Comedy|Crime|Fantasy
11211,22.377852,Hellboy 2004,Action|Adventure|Fantasy|Horror
24238,20.650733,Terminator 3 Rise of the Machines 2003,Action|Adventure|Sci-Fi
61915,20.111741,XMen Origins Wolverine 2009,Action|Sci-Fi|Thriller
44727,18.181971,King Kong 2005,Action|Adventure|Drama|Fantasy|Thriller
36298,17.868540,Chronicles of Riddick The 2004,Action|Sci-Fi|Thriller
28620,17.532425,Transporter The 2002,Action|Crime


,score,title,genres
11933,158.486182,Hulk 2003,Action|Adventure|Sci-Fi
12576,42.157324,Incredible Hulk The 2008,Action|Sci-Fi
54098,26.932292,SpiderMan 3 2007,Action|Adventure|Sci-Fi|Thriller|IMAX
41781,24.299781,Hancock 2008,Action|Adventure|Comedy|Crime|Fantasy
11211,22.377852,Hellboy 2004,Action|Adventure|Fantasy|Horror
24238,20.650733,Terminator 3 Rise of the Machines 2003,Action|Adventure|Sci-Fi
61915,20.111741,XMen Origins Wolverine 2009,Action|Sci-Fi|Thriller
44727,18.181971,King Kong 2005,Action|Adventure|Drama|Fantasy|Thriller
36298,17.868540,Chronicles of Riddick The 2004,Action|Sci-Fi|Thriller
28620,17.532425,Transporter The 2002,Action|Crime
